# Implementação das operações de correlação-cruzada e convolução

Dada uma imagem $f$ e um filtro $w$ de tamanho $a\times b$, a correlação-cruzada para cada pixel $(x,y)$ da imagem é calculada pela fórmula
\begin{equation}
g(x,y)=\sum_{s=0}^{a}\sum_{t=0}^{b}w(s,t)f(x+s-\frac{a}{2}, y+t-\frac{b}{2})
\end{equation}

In [1]:
import numpy as np
import matplotlib.pyplot as plt

def correlation(img, w):
    '''Calcula a correlação cruzada entre a imagem img e o filtro w.
    
       Parâmetros
       ----------
       img : numpy array.
           Imagem a ser filtrada.
       w : numpy array.
           Filtro a ser utilizada na correlação cruzada.
       
       Return
       -------
       img_filtered : numpy array, mesmo tamanho que img
           Imagem filtrada
    '''

    numRows, numCols = img.shape
    fNumRows, fNumCols = w.shape   # fNumRows=a+1 e fNumCols=b+1 (a e b na fórmula acima)

    halfRSize = fNumRows//2        # O operador // retorna a parte inteira da divisão
    halfCSize = fNumCols//2

    # Cria imagem com zeros ao redor da borda
    imgPadded = np.zeros((numRows+fNumRows-1, numCols+fNumCols-1), dtype=img.dtype)
    for row in range(numRows):
        for col in range(numCols):   
            imgPadded[row+halfRSize, col+halfCSize] = img[row, col]
    # A estrutura for aninhada acima pode ser substituída por
    #imgPadded = np.pad(img, ((halfRSize,halfRSize),(halfCSize,halfCSize)), mode='constant')
    
    img_filtered = np.zeros((numRows, numCols))
    for row in range(numRows):
        for col in range(numCols):
            sum_region = 0
            for s in range(fNumRows):
                for t in range(fNumCols):
                    # O índice do array na soma abaixo é diferente do utilizado na fórmula da 
                    # correlação porque estamos utilizando a imagem preenchida com zeros. O 
                    # pixel (0,0)  na imagem preenchida corresponde ao pixel (x+s-a/2, y+t-b/2) 
                    # da imagem original
                    sum_region += w[s, t]*imgPadded[row+s, col+t]
            img_filtered[row, col] = sum_region

            # O loop aninhado acima pode ser substituído por
            # patch = imgPadded[row:row+fNumRows, col:col+fNumCols]
            # img_filtered[row, col] = np.sum(w*patch)
            
    return img_filtered

def convolution(img, w):
    '''Calcula a convolução entre a imagem img e o filtro w.
    
       Parâmetros
       ----------
       img : numpy array.
           Imagem a ser filtrada.
       w : numpy array.
           Filtro a ser utilizada na correlação cruzada.
       
       Return
       -------
       img_filtered : numpy array, mesmo tamanho que img
           Imagem filtrada
    '''
    # Inverte o filtro
    w_inv = w[::-1, ::-1]

    img_filtered = correlation(img, w_inv)

    return img_filtered

## Correlação entre sinal e filtro 1D

In [2]:
# Vamos criar um sinal 1D artificial. Como a função acima foi implementada para arrays 2D,
# nosso sinal 1D será representado por uma matriz de uma única linha
sinal = np.array([[2, 3, 1, 4, 3, 2, 1, 1, 2]])
w = np.array([[1, 2, 3]])
img_filtered = correlation(sinal, w)
print(img_filtered)

[[13. 11. 17. 18. 16. 10.  7.  9.  5.]]


## Comparação entre correlação e convolução

In [3]:
img = np.array([[0, 0, 0, 0, 1, 0, 0, 0, 0]])
w = np.array([[1, 2, 3, 4]])
img_corr_w = correlation(img, w)
w_corr_img = correlation(w, img)
print('img_corr_w:')
print(img_corr_w)
print('w_corr_img:')
print(w_corr_img)
img_conv_w = convolution(img, w)
w_conv_img = convolution(w, img)
print('img_conv_w:')
print(img_conv_w)
print('w_conv_img:')
print(w_conv_img)

img_corr_w:
[[0. 0. 0. 4. 3. 2. 1. 0. 0.]]
w_corr_img:
[[1. 2. 3. 4.]]
img_conv_w:
[[0. 0. 0. 1. 2. 3. 4. 0. 0.]]
w_conv_img:
[[1. 2. 3. 4.]]


## Filtro de média simples aplicado a uma imagem artificial

In [4]:
img = np.array([[1, 2, 3, 2, 1], 
                [2, 1, 3, 2, 1],
                [0, 0, 1, 2, 1],
                [2, 1, 2, 0, 2],
                [3, 2, 1, 2, 3]])
w = np.array([[1, 1, 1],
              [1, 1, 1],
              [1, 1, 1]])
w = w/9.
img_filtered = correlation(img, w)
print(img_filtered)

[[0.66666667 1.33333333 1.44444444 1.33333333 0.66666667]
 [0.66666667 1.44444444 1.77777778 1.77777778 1.        ]
 [0.66666667 1.33333333 1.33333333 1.55555556 0.88888889]
 [0.88888889 1.33333333 1.22222222 1.55555556 1.11111111]
 [0.88888889 1.22222222 0.88888889 1.11111111 0.77777778]]
